In [1]:
import ee
import geemap
from geemap import ml
from sklearn import ensemble
import pandas as pd


# Inicializa la autenticación y la inicialización de Google Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-facuboladerasgee')

In [2]:
roi = ee.FeatureCollection('projects/ee-facuboladerasgee/assets/aoi_pampa')
Map = geemap.Map(center=[0, 0], zoom=2)
Map.add_basemap('SATELLITE')
Map.centerObject(roi)

year = 2021

start = f'{year}-01-01'
end = f'{year}-12-31'

In [3]:
# Sentinel-2 Collection with Cloud Masking
s2_col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(roi) \
    .filterDate(start, end) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

cloud_col = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') \
    .filterBounds(roi) \
    .filterDate(start, end)

def mask_clouds(image):
    cloud_score = ee.Image(cloud_col.filterMetadata('system:index', 'equals', image.get('system:index')).first())
    mask = cloud_score.select('probability').lt(10)
    return image.updateMask(mask)

# Apply cloud masking and compute median
s2_masked = s2_col.map(mask_clouds)
s2_image = s2_masked.median().toFloat().clip(roi)


# Calculate indices (NDVI, MNDWI, NDBI, EVI, SAVI) for Sentinel-2
ndvi = s2_image.normalizedDifference(['B8', 'B4']).rename('NDVI')
mndwi = s2_image.normalizedDifference(['B3', 'B11']).rename('MNDWI')
ndbi = s2_image.normalizedDifference(['B11', 'B8']).rename('NDBI')

evi = s2_image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
    {
        'NIR': s2_image.select('B8'),
        'RED': s2_image.select('B4'),
        'BLUE': s2_image.select('B2')
    }
).rename('EVI')

savi = s2_image.expression(
    '((NIR - RED) / (NIR + RED + 0.5)) * 1.5',
    {
        'NIR': s2_image.select('B8'),
        'RED': s2_image.select('B4')
    }
).rename('SAVI')

fvc = ndvi.expression(
    '((NDVI - NDVI_min) / (NDVI_max - NDVI_min))', {
        'NDVI': ndvi,
        'NDVI_min': 0.2,  # Ajusta según la región de estudio
        'NDVI_max': 0.8   # Ajusta según la región de estudio
    }
).rename('FVC')

# Estimación de LAI (Índice de Área Foliar)
lai = ndvi.expression(
    '-log((1 - FVC) / 0.5)', {
        'FVC': fvc
    }
).rename('LAI')

# Add calculated indices to Sentinel-2 image
s2_image = s2_image.addBands([ndvi, mndwi, ndbi, evi, savi, fvc, lai])

# Add Dynamic World label to the image
dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
    .filterBounds(roi) \
    .filterDate(start, end)

dw_label = dw_col.select('label').median().clip(roi)
s2_image = s2_image.addBands(dw_label.rename('label'))

# Incorporate DEM and slope
dem = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
slope = ee.Terrain.slope(dem).rename('slope').clip(roi)

In [4]:
# Sentinel-1 Data Filtering
def mask_edge(image):
    edge = image.lt(-30.0)
    masked_image = image.mask().And(edge.Not())
    return image.updateMask(masked_image)

def create_filtered_collection(polarization):
    return ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(roi) \
        .filterDate(start, end) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization)) \
        .select(polarization) \
        .map(mask_edge)

img_vv = create_filtered_collection('VV')
img_vh = create_filtered_collection('VH')

# Compute medians for Sentinel-1
s1_vv = img_vv.median().rename('VV').clip(roi)
s1_vh = img_vh.median().rename('VH').clip(roi)

# Combine Sentinel-1 bands
s1_combined = ee.Image.cat([s1_vv, s1_vh])

# Calculate RVI, VV+VH, and VV-VH
rvi = s1_combined.expression(
    '4 * VH / (VV + VH)',
    {
        'VV': s1_combined.select('VV'),
        'VH': s1_combined.select('VH')
    }
).rename('RVI')

vv_plus_vh = s1_combined.expression(
    'VV + VH',
    {
        'VV': s1_combined.select('VV'),
        'VH': s1_combined.select('VH')
    }
).rename('VV_plus_VH')

vv_minus_vh = s1_combined.expression(
    'VV - VH',
    {
        'VV': s1_combined.select('VV'),
        'VH': s1_combined.select('VH')
    }
).rename('VV_minus_VH')

# Add bands to the image
s1_image = s1_combined.addBands([rvi, vv_plus_vh, vv_minus_vh])

# Combine all layers into the final image
image = s2_image.addBands([s1_image, dem.rename('elevation'), slope])

In [5]:
def calculate_texture_metrics(image, band_name, roi, size=5):
    
    # Convertir la banda a tipo int32 (no se realiza normalización adicional)
    band_int = image.select(band_name).toInt32().clip(roi)
    
    # Calcular texturas usando la banda convertida a int32
    glcm = band_int.glcmTexture(size=size)
    
    # Extraer las texturas de interés y renombrarlas
    contrast = glcm.select(f'{band_name}_contrast').rename(f'{band_name}_Contrast')
    correlation = glcm.select(f'{band_name}_corr').rename(f'{band_name}_Correlation')
    entropy = glcm.select(f'{band_name}_ent').rename(f'{band_name}_Entropy')
    inertia = glcm.select(f'{band_name}_inertia').rename(f'{band_name}_Inertia')
    
    # Agregar las bandas de textura a la imagen original
    return image.addBands([contrast, correlation, entropy, inertia])



image = calculate_texture_metrics(image, 'VV', roi)
image = calculate_texture_metrics(image, 'VH', roi)

bands = ['B11', 'B12', 'B8', 'B2', 'B3', 'B4', 'VV', 'VH', 'RVI', 'VV_plus_VH', 'VV_minus_VH', 'elevation', 'slope', 'label', 'EVI','SAVI','LAI', 'FVC','NDVI', 'MNDWI', 'NDBI','VV_Contrast', 'VV_Correlation', 'VV_Entropy' ,'VV_Inertia' ,'VH_Contrast' ,'VH_Correlation' ,'VH_Entropy' ,'VH_Inertia']
image = image.select(bands)

In [56]:
Map.centerObject(roi, 10)
Map.addLayer(image.select('VV_Contrast'), {}, 'VV', False)
Map.addLayer(image.select('VH_Contrast'), {}, 'VH', False)
Map

Map(center=[-32.40021449709678, -58.31767356390013], controls=(WidgetControl(options=['position', 'transparent…

In [57]:
def mask_based_on_label_values(image, values):
    # Cargar la colección de DynamicWorld
    dynamic_world = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
        .filterBounds(roi) \
        .filterDate(start, end) \
        .select('label') \
        .median() \
        .clip(roi)
    
    # Crear una máscara basada en los valores de la lista
    label_mask = dynamic_world.eq(values[0])  # Inicializar con el primer valor
    for value in values[1:]:
        label_mask = label_mask.Or(dynamic_world.eq(value))  # Añadir los demás valores
    
    # Aplicar la máscara de valores al conjunto de datos
    return image.updateMask(label_mask)

# Valores a usar en la máscara (e.g., árboles, vegetación alta, agua, etc.)
values_to_mask = [1]  # Ejemplo de clases de DynamicWorld a incluir

# Obtener los datos GEDI de la altura de dosel
def get_gedi_canopy_height(band_name):
    return ee.ImageCollection('LARSE/GEDI/GEDI02_A_002_MONTHLY') \
        .filterBounds(roi) \
        .filterDate(start, end) \
        .map(lambda image: image.updateMask(
            image.select('degrade_flag').eq(0))) \
        .select(band_name) \
        .median() \
        .toFloat() \
        .clip(roi)

# Obtener la banda de altura de dosel rh95
gedi_rh95 = get_gedi_canopy_height('rh95').rename('rh95')

# Aplicar la máscara basada en los valores seleccionados a los datos GEDI (altura de dosel)
gedi_masked = mask_based_on_label_values(gedi_rh95, values_to_mask)

# Convertir la imagen GEDI enmascarada a float si es necesario
def convert_to_float(image):
    return image.float()

# Convertir la imagen GEDI enmascarada a float
gedi_masked = convert_to_float(gedi_masked)

# Añadir la banda GEDI filtrada (rh95) a la imagen principal
image = image.addBands(gedi_masked.rename('rh95'))

# Opcional: imprimir las bandas de la imagen final para verificar
print(image.bandNames().getInfo())


['B11', 'B12', 'B8', 'B2', 'B3', 'B4', 'VV', 'VH', 'RVI', 'VV_plus_VH', 'VV_minus_VH', 'elevation', 'slope', 'label', 'EVI', 'SAVI', 'LAI', 'FVC', 'NDVI', 'MNDWI', 'NDBI', 'VV_Contrast', 'VV_Correlation', 'VV_Entropy', 'VV_Inertia', 'VH_Contrast', 'VH_Correlation', 'VH_Entropy', 'VH_Inertia', 'rh95']


In [59]:
import time

sample = image.sample(
    scale=10,  # Ajusta la escala según sea necesario para tus datos
    region=roi,
    geometries=True  # Incluir geometría de los puntos
)

# Exportar la tabla de muestra a Google Drive en formato CSV
export_task = ee.batch.Export.table.toDrive(
    collection=sample,
    description='ExportSampleToCSV',
    folder='EE_costaU_CHM',
    fileNamePrefix=f'Datos_RF_2021_10',
    fileFormat='CSV'
)


# Iniciar la tarea de exportación
export_task.start()

    # Esperar a que la tarea de exportación se complete
export_task.status()

    # Verificar el estado de la tarea y mostrar un mensaje de éxito
while export_task.active():
    print('Exportación en progreso...')
    time.sleep(30)  # Esperar 30 segundos antes de verificar el estado nuevamente
    
    if export_task.status()['state'] == 'COMPLETED':
        print(f'Exportación completada con éxito.')
    else:
        print(f'Error en la exportación: {export_task.status()}')

Exportación en progreso...
Error en la exportación: {'state': 'RUNNING', 'description': 'ExportSampleToCSV', 'priority': 100, 'creation_timestamp_ms': 1725282762679, 'update_timestamp_ms': 1725282787752, 'start_timestamp_ms': 1725282767616, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'id': '56FCBKQ2HZSV77YLLKRVKHA2', 'name': 'projects/ee-facuboladerasgee/operations/56FCBKQ2HZSV77YLLKRVKHA2'}
Exportación en progreso...
Error en la exportación: {'state': 'RUNNING', 'description': 'ExportSampleToCSV', 'priority': 100, 'creation_timestamp_ms': 1725282762679, 'update_timestamp_ms': 1725282817752, 'start_timestamp_ms': 1725282767616, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'id': '56FCBKQ2HZSV77YLLKRVKHA2', 'name': 'projects/ee-facuboladerasgee/operations/56FCBKQ2HZSV77YLLKRVKHA2'}
Exportación en progreso...
Error en la exportación: {'state': 'RUNNING', 'description': 'ExportSampleToCSV', 'priority': 100, 'creation_timestamp_ms': 1725282762679, 'update_timestamp_ms': 1725282847753, 's